## Imports

In [1]:
# !pip install jupyterlab

In [37]:
!pip uninstall scikit-learn==0.23.2

^C


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.0
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [4]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "srsoutlook "
    os.environ['KAGGLE_KEY']      = "d2e49883f518122776905d28f5085464"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "datasets/titanic/"

## Load data

In [5]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [6]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [7]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1:
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [8]:
df.head(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [9]:
# CODE HERE get_Title_from_Name funtion
def get_Title_from_Name(name):
    return name.split(',')[1].split('.')[0].strip()

df["Title"]      = df['Name'].map(get_Title_from_Name)
df_test["Title"] = df_test['Name'].map(get_Title_from_Name)

# Exercise 2:
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [10]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [11]:
# CODE HERE

df["Title"] = df['Title'].replace(title_dictionary)
df_test["Title"] =  df_test['Title'].replace(title_dictionary)

# Exercise BONUS:
Try to extract some information from the feature **Ticket**

# Exercise BONUS:
Try to extract some information from the feature **Cabin**

# Preprocessing
For X data:
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [12]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [13]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3:
Create a **ColumnTransformer for Tree Models**. Remember:
- Categorical: Some SimpleImputer -> Some Encoder
- Numerical: Some SimpleImputer -> NO Encoder

In [14]:
num_preprocessing = pipeline.Pipeline(steps=[('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))])


cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  # Some Encoder here. Remember to handle_unknown
    ('ordinal', preprocessing.OrdinalEncoder(handle_unknown='ignore'))# handle_unknown='ignore'
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 4
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [15]:
!pip install catboost

In [16]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [17]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(use_label_encoder=False),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier(verbose=False)
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

# Exercise 5:
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [18]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

for model_name, model in tree_classifiers.items():
    # CODE HERE
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    model.fit(x_train, y_train)
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    y_pred = model.predict(x_val)
    print(f"Accuracy of {model_name}:          ", metrics.accuracy_score(y_val, y_pred)*100)
    print(f"Balanced accuracy of {model_name}: ", metrics.balanced_accuracy_score(y_val, y_pred)*100)
    print('-----' * 5)

Accuracy of Decision Tree:           76.53631284916202
Balanced accuracy of Decision Tree:  74.96706192358367
-------------------------
Accuracy of Extra Trees:           77.09497206703911
Balanced accuracy of Extra Trees:  74.34123847167325
-------------------------
Accuracy of Random Forest:           77.6536312849162
Balanced accuracy of Random Forest:  75.06587615283267
-------------------------
Accuracy of AdaBoost:           79.88826815642457
Balanced accuracy of AdaBoost:  77.96442687747036
-------------------------
Accuracy of Skl GBM:           84.91620111731844
Balanced accuracy of Skl GBM:  82.59552042160738
-------------------------
Accuracy of Skl HistGBM:           81.00558659217877
Balanced accuracy of Skl HistGBM:  78.60342555994731
-------------------------
[14:27:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed fr

# Exercise 6:
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [19]:
skf = model_selection.StratifiedKFold(
    # CODE HERE
    n_splits=10, shuffle=True, random_state=0
)

for model_name, model in tree_classifiers.items():
    # CODE HERE
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    model.fit(x_train, y_train)
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    y_pred_cv = model_selection.cross_val_predict(model, x, y, cv=skf)
    y_score = model_selection.cross_val_score(model, x, y, cv=skf)
    print(f'Lower score for {model_name}: {min(y_score)}, max: {max(y_score)}, mean: {np.mean([min(y_score), max(y_score)])}')
    print('-----------' *5)

Lower score for Decision Tree: 0.7078651685393258, max: 0.8444444444444444, mean: 0.7761548064918851
-------------------------------------------------------
Lower score for Extra Trees: 0.7640449438202247, max: 0.8539325842696629, mean: 0.8089887640449438
-------------------------------------------------------
Lower score for Random Forest: 0.7640449438202247, max: 0.8539325842696629, mean: 0.8089887640449438
-------------------------------------------------------
Lower score for AdaBoost: 0.6853932584269663, max: 0.8555555555555555, mean: 0.7704744069912609
-------------------------------------------------------
Lower score for Skl GBM: 0.7528089887640449, max: 0.898876404494382, mean: 0.8258426966292134
-------------------------------------------------------
Lower score for Skl HistGBM: 0.7865168539325843, max: 0.8539325842696629, mean: 0.8202247191011236
-------------------------------------------------------
[14:28:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release

# Exercise 7
Train **with all data** the best model

In [20]:
best_model = GradientBoostingClassifier() # Select your best model

# Train with all data your best model
x_prepro      = tree_prepro.fit_transform(x)# ONLY FIT THE PREPROCESSOR ON TRAIN TO AVOID DATA LEAKS !!!
x_test_prepro = tree_prepro.transform(x_test)
best_model.fit(x_prepro, y)

GradientBoostingClassifier()

# Exercise 8
With your best model, generate the predicitions for test data (x_test)

In [21]:
test_pred = best_model.predict(x_test_prepro) # Get the predictions for x_test

# Exercise 9

Submit to kaggle using the kaggle API. And send us your score. You can try to improve it.

In [22]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [23]:
sub.to_csv("sub_skl_gbm.csv")

In [24]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


# Exercise BONUS

Knowing how to export your models is very important for putting models in production. Try to
- Export and Load the ColumTransformer in pickle
- Export and Load the ColumTransformer in joblib
- Export and load the Pipeline